In [1]:
from keras.models import Sequential,Model
from keras.layers import Activation
from keras.layers.core import Dense,Flatten,Dropout
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
from keras.optimizers import Adam, RMSprop , SGD
import keras 
import keras.backend as B
import numpy as np
import keras.applications as A
import matplotlib.pyplot as plt
%matplotlib inline

import os
from keras import regularizers

Using TensorFlow backend.


In [2]:
os.listdir("dataset2-master")

['.DS_Store', 'images', 'labels.csv']

In [3]:
cd "dataset2-master/"

C:\Users\ARPIT JAIN\Desktop\dataset2-master


In [13]:
train_path = "images/TRAIN"
test_path = "images/TEST"

In [5]:
inputs = (240,320,3)

In [6]:
# train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224),classes=['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL'],batch_size=50)
# test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224),classes=['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL'],batch_size=50)

In [7]:
vgg = A.vgg16.VGG16(weights='imagenet',include_top=False, input_shape=inputs)
#vgg.summary()

58892288/58889256 [==============================] - 33s 1us/step


In [8]:
model = Sequential()
for layer in vgg.layers[:-4]:
  layer.trainable=False

In [9]:
for layer in vgg.layers:
  print(layer,layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x000001DCBD20A6D8> False
<keras.layers.convolutional.Conv2D object at 0x000001DCC55E3E10> False
<keras.layers.convolutional.Conv2D object at 0x000001DCC55E3F98> False
<keras.layers.pooling.MaxPooling2D object at 0x000001DCC5630278> False
<keras.layers.convolutional.Conv2D object at 0x000001DCC5608D68> False
<keras.layers.convolutional.Conv2D object at 0x000001DCC5649470> False
<keras.layers.pooling.MaxPooling2D object at 0x000001DCC5669748> False
<keras.layers.convolutional.Conv2D object at 0x000001DCC5669F60> False
<keras.layers.convolutional.Conv2D object at 0x000001DCC56A4128> False
<keras.layers.convolutional.Conv2D object at 0x000001DCC56C3400> False
<keras.layers.pooling.MaxPooling2D object at 0x000001DCC56DD358> False
<keras.layers.convolutional.Conv2D object at 0x000001DCC56DDC88> False
<keras.layers.convolutional.Conv2D object at 0x000001DCC5713160> False
<keras.layers.convolutional.Conv2D object at 0x000001DCC5733470> False
<ker

In [10]:
# Add the vgg convolutional base model
model.add(vgg)
 
# Add new layers
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(32, activation='relu' ))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(16, activation='relu' ))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(8, activation='relu' ))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(4, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 10, 512)        14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 35840)             0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 35840)             143360    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1146912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
__________

In [11]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest', validation_split=0.25)


validation_datagen = ImageDataGenerator(rescale=1./255 )
 
# Change the batchsize according to your system RAM
train_batchsize = 20
val_batchsize = 20

In [14]:
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(240, 320),
        batch_size=train_batchsize,
        class_mode='categorical', subset = "training")
 
validation_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(240, 320),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False,
    subset='validation')

Found 7470 images belonging to 4 classes.
Found 2487 images belonging to 4 classes.


In [15]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4),
              metrics=['acc'])

In [ ]:
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples//train_generator.batch_size ,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples//validation_generator.batch_size,
      verbose=1)

Epoch 1/50
 72/373 [====>.........................] - ETA: 2:15:26 - loss: 1.7179 - acc: 0.2924

In [ ]:
model.save("model_2.h5")

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

In [ ]:
print("The best Training accuracy {}".format(max(acc)*100))
print("The best validation accuracy {} ".format(max(val_acc)*100))